In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import  numpy as np
import dask.dataframe as dd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import dill
import datetime
import os

In [2]:
#
df_target = pd.read_csv("train_data/train_target.csv")
df_target.head()

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [3]:
# Загрузим данные из файлов в датафрейм...
# а также  все фичи развернем в столбцы, так как многочисленные эксперименты показали, что иначе т ребуемой точности не добиться...
def read_and_prepare():
    ohe = OneHotEncoder(sparse_output=False)
    df_final=pd.DataFrame()
    for i in range(0, 12):
        file_path = "train_data/train_data_"+str(i)+".pq"
        df = dd.read_parquet(file_path)     
        df = df.compute()
        print("Количество пропусков в файле ", file_path," ", df.columns.isna().sum())
        all_features = df.columns.to_list()
        all_features.remove("id")
        encoded_data  = ohe.fit_transform(df[all_features])
        encoded_data = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out())
        #присоединим к датафрейму закодированные фичи
        df = pd.concat([df, encoded_data], axis=1) 
        #удалим все исходные фичи
        df = df.drop(all_features, axis=1)
        #сгруппируем по id клиента
        df_agg = df.groupby('id').sum()   
        #смержим с таргетом
        df_agg_new = pd.merge(df_agg, df_target, on='id', how='inner')
        #присоединим полученный датафрейм к итоговому
        df_final =pd.concat( [df_agg_new, df_final], axis=0)
    return df_final   

In [4]:
df_final = read_and_prepare()

model =  XGBClassifier(n_jobs=-1, class_weight='balanced')

pipe = Pipeline(steps=[
           ('classifier',  model)
    ])


pipe.fit(df_final, df_final['flag'])

with open("ML_Final.pkl", 'wb') as file:
    dill.dump(
              { "model": model,
                    "metadata": {
                        "name": "My ML Final",
                        "author": "Elena Petrova",
                        "version": 1,
                        "date": datetime.datetime.now(),
                        "type": type(pipe.named_steps["classifier"]).__name__,
                    }
                },
    file)

Количество пропусков в файле  train_data/train_data_0.pq   0
Количество пропусков в файле  train_data/train_data_1.pq   0
Количество пропусков в файле  train_data/train_data_2.pq   0
Количество пропусков в файле  train_data/train_data_3.pq   0
Количество пропусков в файле  train_data/train_data_4.pq   0
Количество пропусков в файле  train_data/train_data_5.pq   0
Количество пропусков в файле  train_data/train_data_6.pq   0
Количество пропусков в файле  train_data/train_data_7.pq   0
Количество пропусков в файле  train_data/train_data_8.pq   0
Количество пропусков в файле  train_data/train_data_9.pq   0
Количество пропусков в файле  train_data/train_data_10.pq   0
Количество пропусков в файле  train_data/train_data_11.pq   0
[21:54:14] WARNING: ../src/learner.cc:767: 
Parameters: { "class_weight" } are not used.

